# **DAILY MACRO RELATIVE VALUE TRADE FRAMEWORK**
### Author:  Deborah Akintoye
#### **Purpose:** Convert one macro headline into a backtestable relative-value trade with clear horizon, factor sensitivities, execution logic, and weekly performance tracking.

---


## 1. Trade Metadata & Idea Structuring

**Date:    19-Jan-2026**

**News Source / Catalyst:** EU readies €93bn tariffs in retaliation for Trump’s Greenland threat

**Markets (s) Impacted:** FX (G10/EM), FI (Euro-Periphery vs. EM spreads)

**Trade Taxonomy:** Macro RV Baskets / Policy Divergence

**Trade Idea (1 sentence):**
> Long a basket of INR, IDR, and PHP (Resilient Asian Carry) against a Short basket of NOK, SEK, and GBP (Greenland/Aric Tariff Targets).

**Client/Desk Pitch (30 seconds):**
> We are shorting the "Western Alliance Fissure." While Trump targets the UK and Scandis with 10% tariffs over the Greenland dispute, and the EU prepares €93bn in retaliation, the structural growth stories in South/Southeast Asia remain decoupled from Arctic geopolitics. This trade captures the "Geopolitical Risk Premium" shift from the North Atlantic to a stable, carry-rich Asian basket.

**Why Now:**

The "Davos Countdown." With the Feb 1 tariff deadline looming, we expect a volatility "squeeze" in Scandi and UK markets as negotiators head to Switzerland.

---

## 2. Macro Hypothesis & Instruments

### **Macro Hypothesis:**

The "Greenland Discount" will metastasize across the currencies of countries explicitly named in Trump's tariff threat (UK, Norway, Sweden). Conversely, India, Indonesia, and the Philippines are net energy importers that benefit from trade-war-induced oil price suppression and have high real rates that provide a "carry buffer."

**Long Leg (Tickers + Rationale):**
- INR (USDINR=X), IDR (USDIDR=X), PHP (USDPHP=X); Low correlation to Arctic security; benefit from structural capital inflows; high domestic demand insulation

**Short Leg (Tickers + Rationale):**
- NOK (USDNOK=X), SEK (USDSEK=X), GBP (GBPUSD=X); Direct tariff targets; NOK/SEK liquidity risk; GBP vulnerability to trade negotiations

**Weights:** Beta-Neutral / Vol-Scaled

**Allocation**: We apply a $3:1$ ratio on notional. Because EM FX volatility is artificially suppressed by central bank intervention (RBI, BI), we need higher notional in the Long leg to match the high-beta daily swings of the Scandis.

---

## 3. Factor Sensitivity & Horizon Alignment

| Factor                | Strength (S/M/W) | Notes |
|----------------------|------------------|-------|
| USD / FX             |        M          |  The basket is USD-neutral; we are trading the Cross-Rate proxy     |
| Oil Beta             |        H          |   Long leg = Short Oil; Short leg = Long Oil (specifically NOK)    |
| Rates Duration       |        M          |   Positive carry (EM yields > Scandi/UK yields)    |
| Growth vs Value      |        M          |   Favors domestic service-led growth (EM) over export-led (Scandi)    |
| Risk-On/Off          |        M          |   Defensive in a localsed "Atlantic Trade War" scenario    |
| China/EM Beta        |        S          |   Long leg captures the "Neutral EM" safety trade    |
| Volatility Regime    |        S          |  Betting on a vol-spike in Northern Europe vs. stability in Asia     |

**Catalyst Type:** Policy / Regime Shift

**Realisation Window:** 14 - 21 days (leading to 01-Feb Implementation)

**Holding Period Selected:**  Tactical (Davos meeting duration)

**Alignment Score:** Good

---


## 4. Risk Controls & Stop Logic

**Position Sizing:** 5.0% of portfolio notional (Aggregate basket)

**Stop-Loss:** -2.8% on the basket spread

**Take-Profit (optional):**  7.5% (Triggered if EU formally activates the "Anti-Coercion Instrument")

**Instrument Contamination:**   We avoid EUR and CHF in the short leg because they act as safe-havens during European panics, which would hedge our short unfairly. We use USD as the denominator for all to maintain a "Synthetic Cross" structure.

**Failure Mode (tick):**
- [ ] Time
- [ ] Price
- [X] Information

---


# **PYTHON IMPLEMENTATION**

In [45]:
# --- Python Imports ---

import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

pd.set_option('display.float_format', lambda x: f'{x:.4f}')


In [73]:
# --- Parameters ---

long_tickers = ["USDINR=X", "USDIDR=X", "USDPHP=X"]     # EDIT HERE
short_tickers = ["USDNOK=X", "USDSEK=X", "GBPUSD=X"]                    # EDIT HERE

start_date = "2020-01-01"                         # Backtest window
holding_period = 14                               # days
stop_loss = -0.028                                 # -2% stop
take_profit = 0.075                                # optional e.g. +0.03 for +3%

# --- Weightings ---

weights_long = {"INRUSD=X": 0.33, "IDRUSD=X": 0.33, "PHPUSD=X": 0.34}
weights_short = {"USDNOK=X": 0.15,  "USDSEK=X": 0.15,   "GBPUSD=X": -0.20}


In [74]:
# --- Data Download ---

tickers = long_tickers + short_tickers
data = yf.download(tickers, start=start_date)["Close"]

# Drop empty cols & make sure no NA

data = data.dropna(how='all').ffill()

returns = data.pct_change().dropna()



/var/folders/nn/rjk_sthj67g1rcv7qxd8_14m0000gn/T/ipykernel_9158/1942310636.py:4: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(tickers, start=start_date)["Close"]
[*********************100%***********************]  6 of 6 completed


In [75]:
# --- Spread Returns ---

long_returns = returns[long_tickers].mean(axis=1)
short_returns = returns[short_tickers].mean(axis=1)

strategy_returns = long_returns - short_returns


In [76]:
# --- Backtest Simulation ---

trade_pnl = []
trade_dates = []

dates = strategy_returns.index
i = 0
n = len(strategy_returns)

while i < n - holding_period:
    pnl = 0.0
    stopped = False
    entry_date = dates[i]
    
    for j in range(holding_period):
        daily_ret = strategy_returns.iloc[i + j]
        pnl = (1 + pnl) * (1 + daily_ret) - 1
        
        if pnl <= stop_loss:
            trade_pnl.append(pnl)
            trade_dates.append(entry_date)
            stopped = True
            break
        
        if take_profit is not None and pnl >= take_profit:
            trade_pnl.append(pnl)
            trade_dates.append(entry_date)
            stopped = True
            break
    
    if not stopped:
        trade_pnl.append(pnl)
        trade_dates.append(entry_date)
    
    i += holding_period

trade_results = pd.DataFrame({
    "Trade Date": trade_dates,
    "PnL": trade_pnl
})

trade_results.tail()


,Trade Date,PnL
107,2025-10-03,-0.0074
108,2025-10-23,0.0087
109,2025-11-12,-0.0000
110,2025-12-02,0.0060
111,2025-12-22,0.0104


In [77]:
# --- Performance Summary ---

summary = {
    "Total Trades": len(trade_results),
    "Win Rate": (trade_results["PnL"] > 0).mean(),
    "Average PnL": trade_results["PnL"].mean(),
    "Best Trade": trade_results["PnL"].max(),
    "Worst Trade": trade_results["PnL"].min(),
    "Stop-Out Frequency": (trade_results["PnL"] <= stop_loss).mean()
}

pd.Series(summary)


Total Trades         112.0000
Win Rate               0.5179
Average PnL            0.0016
Best Trade             0.1161
Worst Trade           -0.0378
Stop-Out Frequency     0.0536
dtype: float64

## Interpretation & Trader Notes

**Absolute Metrics:**
- Total Trades: 
- Win Rate:
- Avg PnL:
- Best:
- Worst: 
- Stop-Outs: 

**What went right:**
- 

**What went wrong:**
- 

**Factor that actually drove results:**
- 

**Would I trade this live?** (Yes / No and why)
>

---

# **BLOOMBERG-STYLE TRADE BLOTTER**

In [78]:
import uuid
from datetime import datetime

def generate_trade_id(prefix="RV"):
    return f"{prefix}-{uuid.uuid4().hex[:8].upper()}"


In [79]:
# --- Computing Synthetic Basket Price Series ---

basket_long = (data[long_tickers] * weights_long).sum(axis=1)
basket_short = (data[short_tickers] * weights_short).sum(axis=1)

basket_spread = basket_long - basket_short


In [80]:
# --- Backtesting Loop Modified for Blotter ---

blotter_rows = []

i = 0
n = len(strategy_returns)
dates = strategy_returns.index

while i < n - holding_period:
    entry_date = dates[i]
    exit_date = dates[min(i + holding_period - 1, n - 1)]
    
    entry_price = basket_spread.loc[entry_date]
    
    pnl = 0.0
    stopped = False
    trade_status = "CLOSED"
    
    for j in range(holding_period):
        daily_ret = strategy_returns.iloc[i + j]
        pnl = (1 + pnl) * (1 + daily_ret) - 1
        
        if pnl <= stop_loss:
            exit_date = dates[i + j]
            trade_status = "STOPPED"
            stopped = True
            break
    
    if not stopped and take_profit is not None and pnl >= take_profit:
        exit_date = dates[i + j]
        trade_status = "TAKE_PROFIT"
    
    exit_price = basket_spread.loc[exit_date]
    holding_days = (exit_date - entry_date).days
    
    row = {
        "TradeID": generate_trade_id(prefix="RV"),
        "Timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        "Strategy": "Macro RV Spread",
        "Taxonomy": trade_taxonomy if 'trade_taxonomy' in globals() else "RV Macro Spread",
        "LongLeg": ", ".join(long_tickers),
        "ShortLeg": ", ".join(short_tickers),
        "EntryDate": entry_date.strftime("%Y-%m-%d"),
        "ExitDate": exit_date.strftime("%Y-%m-%d"),
        "EntryPrice": entry_price,
        "ExitPrice": exit_price,
        "GrossPnL(%)": pnl * 100,
        "StopLoss(%)": stop_loss * 100,
        "TakeProfit(%)": take_profit * 100 if take_profit else None,
        "Status": trade_status,
        "HoldingPeriod": holding_days,
        "Notes": "",  # Optional notes field
    }
    blotter_rows.append(row)
    
    i += holding_period

blotter = pd.DataFrame(blotter_rows)
blotter.tail()


,TradeID,Timestamp,Strategy,Taxonomy,LongLeg,ShortLeg,EntryDate,ExitDate,EntryPrice,ExitPrice,GrossPnL(%),StopLoss(%),TakeProfit(%),Status,HoldingPeriod,Notes
107,RV-75E2DD96,2026-01-19 06:49:21,Macro RV Spread,RV Macro Spread,"USDINR=X, USDIDR=X, USDPHP=X","USDNOK=X, USDSEK=X, GBPUSD=X",2025-10-03,2025-10-22,-2.6364,-2.6557,-0.7398,-2.8000,7.5000,CLOSED,19,
108,RV-A4796FA3,2026-01-19 06:49:21,Macro RV Spread,RV Macro Spread,"USDINR=X, USDIDR=X, USDPHP=X","USDNOK=X, USDSEK=X, GBPUSD=X",2025-10-23,2025-11-11,-2.6476,-2.6822,0.8723,-2.8000,7.5000,CLOSED,19,
109,RV-A642919B,2026-01-19 06:49:21,Macro RV Spread,RV Macro Spread,"USDINR=X, USDIDR=X, USDPHP=X","USDNOK=X, USDSEK=X, GBPUSD=X",2025-11-12,2025-12-01,-2.6610,-2.6698,-0.0023,-2.8000,7.5000,CLOSED,19,
110,RV-E5EBFBEC,2026-01-19 06:49:21,Macro RV Spread,RV Macro Spread,"USDINR=X, USDIDR=X, USDPHP=X","USDNOK=X, USDSEK=X, GBPUSD=X",2025-12-02,2025-12-19,-2.6727,-2.6471,0.5988,-2.8000,7.5000,CLOSED,17,
111,RV-25B0405B,2026-01-19 06:49:21,Macro RV Spread,RV Macro Spread,"USDINR=X, USDIDR=X, USDPHP=X","USDNOK=X, USDSEK=X, GBPUSD=X",2025-12-22,2026-01-12,-2.6422,-2.6268,1.0406,-2.8000,7.5000,CLOSED,21,


In [81]:
blotter.to_csv("trade_blotter.csv", index=False)